In [41]:
#import all necessary libraries 
import csv 
import datetime
import boto3
import pandas as pd
from pyspark import SparkContext, SparkConf
from datetime import timedelta
from pyspark.sql import *
from pyspark.sql .types import *

In [47]:
#DeclareConstants
from dams import FileSystem, S3Dataset
INPUT_BUCKET_NAME="daimler-phase1"
INPUT_DATA_PREFIX="01-dailic00162_"
start_date_file=datetime.date(2017, 1, 1)
end_date_file=datetime.date(2018, 1, 1)
current_date = start_date_file
DATE_FORMAT = '%Y-%m-%d'
OUTPUT_DATA_FOLDER="SplitData"
INPUT_DATA_FOLDER="POC"
INPUT_DATA_PREFIX="01-dailic00162_"
SEARCH_MOB_STRING="Mobile Web"
SEARCH_DES_STRING="Desktop"
INPUT_DATA_TYPE=".tsv"
INPUT_DATA_SEPARATOR="\t"
INPUT_DATA_HAS_HEADER=None

In [75]:
#Appends the correct file 
def get_mob_partition_key(date):
    return SEARCH_MOB_STRING+date.strftime(DATE_FORMAT)+".csv"

def format_s3_partition_key(key):
    return OUTPUT_DATA_FOLDER+"/"+key

def get_desk_paritition_key(date):
    return SEARCH_DES_STRING+date.strftime(DATE_FORMAT)+".csv"


current_partition = get_mob_partition_key(current_date)
counter= 0

In [76]:
#Deprecated function - of no use - transformed the same logic to pyspark
def mobile(filem):
    print(filem)
    with open("output.csv", "w",) as output:
        writer = csv.writer(output ,quoting=csv.QUOTE_NONE,escapechar='}',dialect='excel')
        for row in filem:
                print([row])
                if row in SEARCH_MOB_STRING :
                    print([row])
                    writer.writerow([row])



In [77]:
#Actual mapping happens here and Lines has to encoded to utf-8 to ensure there is no ascii bad characters 
def mapper(line):
    key = "desktop"
    if SEARCH_MOB_STRING in line:
        key = "mobile"
    return (key, line.encode("utf-8").split("\t"))

In [79]:
#Download the s3 data and do the mapping and store it back in local hard disk 
def download():
s3=S3Dataset('daimler-phase1')
sc = SparkContext.getOrCreate()
while current_date<end_date_file:
        s3.download(INPUT_DATA_FOLDER+"/"+INPUT_DATA_PREFIX+current_date.strftime(DATE_FORMAT)+INPUT_DATA_TYPE, "to_split_temp"+INPUT_DATA_TYPE)
        data = sc.textFile("to_split_temp"+INPUT_DATA_TYPE)
        mapped_data = data.map(mapper)
        with open("/home/ab/pyspark/output/"+get_mob_partition_key(current_date), 'wb') as csvfile:
            with open ("/home/ab/pyspark/output/"+get_desk_paritition_key(current_date),'wb')as csvfiledesktop:
                writer = csv.writer(csvfile, delimiter=",", escapechar='}', quoting=csv.QUOTE_NONE)
                writerdesk=csv.writer(csvfiledesktop, delimiter=",", escapechar='}', quoting=csv.QUOTE_NONE)
                for k,v in mapped_data.collect():
                    if k == "mobile":
                        writer.writerow([v])
                    elif k=="desktop":
                        writerdesk.writerow([v])
        #s3.upload("/home/ab/pyspark/output/"+current_partition, format_s3_partition_key(current_partition))
        current_date = current_date + timedelta(days=1)
        print(current_date)
        counter+=1


2017-06-16
2017-06-17
2017-06-18
2017-06-19
2017-06-20
2017-06-21
2017-06-22
2017-06-23
2017-06-24
2017-06-25
2017-06-26
2017-06-27
2017-06-28
2017-06-29
2017-06-30
2017-07-01
2017-07-02
2017-07-03
2017-07-04
2017-07-05
2017-07-06
2017-07-07
2017-07-08
2017-07-09
2017-07-10
2017-07-11
2017-07-12
2017-07-13
2017-07-14
2017-07-15
2017-07-16
2017-07-17
2017-07-18
2017-07-19
2017-07-20
2017-07-21
2017-07-22
2017-07-23
2017-07-24
2017-07-25
2017-07-26
2017-07-27
2017-07-28
2017-07-29
2017-07-30
2017-07-31
2017-08-01
2017-08-02
2017-08-03
2017-08-04
2017-08-05
2017-08-06
2017-08-07
2017-08-08
2017-08-09
2017-08-10
2017-08-11
2017-08-12
2017-08-13
2017-08-14
2017-08-15
2017-08-16
2017-08-17
2017-08-18
2017-08-19
2017-08-20
2017-08-21
2017-08-22
2017-08-23
2017-08-24
2017-08-25
2017-08-26
2017-08-27
2017-08-28
2017-08-29
2017-08-30
2017-08-31
2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05
2017-09-06
2017-09-07
2017-09-08
2017-09-09
2017-09-10
2017-09-11
2017-09-12
2017-09-13
2017-09-14

In [82]:
#upload the localhard disk to s3
def upload():
for file in FileSystem.list_files("/home/ab/pyspark/output"):
    print(file)
    s3.upload("/home/ab/pyspark/output/"+file, format_s3_partition_key(file))
    #FileSystem.delete_file(current_partition)


Desktop2017-10-06.csv
Desktop2017-09-28.csv
Desktop2017-10-12.csv
Mobile Web2017-07-23.csv
Desktop2017-04-10.csv
Mobile Web2017-01-28.csv
Mobile Web2017-10-15.csv
Desktop2017-03-28.csv
Mobile Web2017-08-13.csv
Desktop2017-12-10.csv
Mobile Web2017-11-20.csv
Desktop2017-06-22.csv
Desktop2017-11-05.csv
Mobile Web2017-09-29.csv
Mobile Web2017-07-14.csv
Mobile Web2017-02-27.csv
Desktop2017-03-14.csv
Mobile Web2017-06-07.csv
Mobile Web2017-02-17.csv
Desktop2017-02-03.csv
Mobile Web2017-01-22.csv
Desktop2017-09-29.csv
Desktop2017-09-08.csv
Mobile Web2017-01-26.csv
Desktop2017-12-16.csv
Mobile Web2017-05-24.csv
Desktop2017-01-31.csv
Mobile Web2017-05-25.csv
Mobile Web2017-08-25.csv
Mobile Web2017-06-29.csv
Desktop2017-07-03.csv
Desktop2017-06-29.csv
Mobile Web2017-03-21.csv
Mobile Web2017-11-18.csv
Desktop2017-02-11.csv
Mobile Web2017-04-16.csv
Mobile Web2017-03-25.csv
Mobile Web2017-09-17.csv
Mobile Web2017-09-12.csv
Desktop2017-10-03.csv
Desktop2017-06-28.csv
Desktop2017-07-30.csv
Desktop201

Mobile Web2017-09-16.csv
Desktop2017-12-24.csv
Desktop2017-05-10.csv
Mobile Web2017-12-08.csv
Desktop2017-09-26.csv
Desktop2017-07-16.csv
Desktop2017-04-16.csv
Mobile Web2017-05-31.csv
Desktop2017-10-17.csv
Desktop2017-12-11.csv
Desktop2017-02-14.csv
Mobile Web2017-03-09.csv
Desktop2017-08-20.csv
Desktop2017-02-22.csv
Desktop2017-04-27.csv
Desktop2017-03-29.csv
Desktop2017-02-26.csv
Mobile Web2017-04-14.csv
Desktop2017-07-10.csv
Mobile Web2017-11-01.csv
Mobile Web2017-07-05.csv
Mobile Web2017-08-04.csv
Desktop2017-07-06.csv
Desktop2017-10-28.csv
Mobile Web2017-10-10.csv
Desktop2017-03-11.csv
Mobile Web2017-05-14.csv
Mobile Web2017-03-24.csv
Mobile Web2017-05-18.csv
Desktop2017-01-16.csv
Desktop2017-03-16.csv
Mobile Web2017-01-21.csv
Mobile Web2017-02-22.csv
Mobile Web2017-12-12.csv
Mobile Web2017-08-18.csv
Mobile Web2017-11-04.csv
Desktop2017-12-09.csv
Mobile Web2017-12-06.csv
Desktop2017-11-07.csv
Desktop2017-12-07.csv
Desktop2017-07-05.csv
Mobile Web2017-05-01.csv
Desktop2017-11-28.c

Desktop2017-05-06.csv
Desktop2017-06-04.csv
Desktop2017-11-27.csv
Desktop2017-11-10.csv


In [68]:
#testdata - always use these functions instead of testing all the functions 
data = sc.textFile("to_split_temp"+INPUT_DATA_TYPE)
mapped_data = data.map(mapper)

In [70]:
#testdata - always use these functions instead of testing all the functions 
mapped_data.collect()[:10]
    

[('mobile',
  ['en-us',
   '717426878',
   '559',
   '375',
   '32',
   '',
   'm1.com.sg:m1 limited',
   '',
   '',
   '0',
   '',
   '0',
   '0',
   '',
   'JS-1.5',
   '1',
   '4',
   'N',
   '200',
   '',
   '2',
   '1.000000000000',
   '',
   '1484324934',
   '',
   '1',
   '2017-01-14 00:28:54',
   'm1.com.sg',
   '',
   '0',
   '',
   '',
   'sg:COS:mb:en:Login:',
   'sg:COS:mb:en:Login',
   'Login',
   '',
   'COS',
   'Mobile Web',
   'sg',
   'en',
   'mb',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   'anonymous',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
 